# 2-node extended repressilator

This repressilator includes additional "steps" - C4 and C14 signalling molecules that act as intermediates between repressors.

In [ ]:
pip install -e .

In [ ]:
import loica as lc
import matplotlib.pyplot as plt
import networkx as nx
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Diff, continuous AiiA

#### code

In [ ]:
def growth_rate(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 1)

def biomass(t):
    return lc.gompertz(t, 0.01, 1, 1, 1)

metab = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
genetic_n = lc.GeneticNetwork()

In [ ]:
strain = lc.Strain('Strain', genetic_n, metab)

In [ ]:
# Transcription factors
laci = lc.Regulator(name='LacI', degradation_rate=1.02, init_concentration=10**-7)
tetr = lc.Regulator(name='TetR', degradation_rate=1.02)
ci = lc.Regulator(name='cI', degradation_rate=1.02)

# Signalling molecules
c4 = lc.Regulator(name='C4', diffusion_rate=180)
c14 = lc.Regulator(name='C14', diffusion_rate=126)

genetic_n.add_regulator([tetr, laci, ci, c4, c14])

In [ ]:
# option with producers
rhli = lc.Regulator(name='RhlI', degradation_rate=1.02)
cini = lc.Regulator(name='CinI', degradation_rate=1.02)
genetic_n.add_regulator([rhli, cini])

In [ ]:
aiia = lc.Regulator(name='AiiA', degradation_rate=1.02)
genetic_n.add_regulator(aiia)

In [ ]:
mcherry = lc.Reporter(name='mCherry', degradation_rate=1.02, color='red') 
cyan = lc.Reporter(name='CFP', degradation_rate=1.02, color='blue') 
mven = lc.Reporter(name='mVenus', degradation_rate=1.02, color='yellow') 

genetic_n.add_reporter([mcherry, cyan, mven])

In [ ]:
# option with producers

#First extended node
laci_not_cini = lc.Hill1(name='Plac', input=laci, output=cini, alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)
cini_c14 = lc.Producer(name='CinI', enzyme=cini, product=c14, k2=60)
c14_tetr = lc.Hill1(name='Pcin', input=c14, output=[tetr, mcherry], alpha=[2.15*10**-8, 7.35*10**-8], K=3.41*10**-6, n=1.87)

# Second node
tetr_not_ci = lc.Hill1(name='Ptet', input=tetr, output=[ci, mven], alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)

# Third extended node
ci_not_rhli = lc.Hill1(name='PcI', input=ci, output=rhli, alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)
rhli_c4 = lc.Producer(name='RhlI', enzyme=rhli, product=c4, k2=960)
c4_laci = lc.Hill1(name='Prhl', input=c4, output=[laci, cyan], alpha=[0, 2.15*10**-8], K=4.68*10**-5, n=0.64)

genetic_n.add_operator([c4_laci, laci_not_cini, c14_tetr, ci_not_rhli, tetr_not_ci])
genetic_n.add_impactor([cini_c14, rhli_c4])

In [ ]:
lactonase = lc.Degrader(name='Lactonase', enzyme=aiia, substrate=[c4, c14], Km=[0.00511, 0.00143], k2=[1.35*10**5, 7.28*10**4])
genetic_n.add_impactor(lactonase)

# continuous expression of AiiA, one copy
j23100 = lc.Source(output=aiia, rate=2.56*10**-7, name='J23100')
genetic_n.add_operator(j23100)

In [ ]:
plt.figure(figsize=(3.5,3.5), dpi=100)
genetic_n.draw() #pos=nx.planar_layout

In [ ]:
sample = lc.Sample(strain=strain)
assay = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='Extended repressilator',
                description='Simulated expression of extended repressilator strain'
                )

In [ ]:
assay.run(substeps=100, mode="track_all")

#### results

In [ ]:
m = assay.measurements
fig, ax = plt.subplots()
fig1, ax1 = plt.subplots()
fig2, ax2 = plt.subplots()
fig3, ax3 = plt.subplots()
m[m.Signal=='mCherry in Strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
m[m.Signal=='CFP in Strain'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax)
m[m.Signal=='mVenus in Strain'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax)
m[m.Signal=='LacI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='TetR in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='cI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='C4 in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='C14 in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='RhlI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax3)
m[m.Signal=='CinI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax3)
ax.legend(['mCherry', 'CFP', 'mVenus'])
ax.set_xlabel('Time (h)')
ax.set_ylabel('Concentration (M)')
ax1.legend(['LacI', 'TetR', 'cI'])
ax1.set_xlabel('Time (h)')
ax1.set_ylabel('Concentration (M)')
ax2.legend(['C4', 'C14'])
ax2.set_xlabel('Time (h)')
ax2.set_ylabel('Concentration (M)')
ax3.legend(['RhlI', 'CinI'])
ax3.set_xlabel('Time (h)')
ax3.set_ylabel('Concentration (M)')

fig4, ax4 = plt.subplots()
m[m.Signal=='mCherry in Strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax4)
m[m.Signal=='CFP in Strain'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax4)
ax4.legend(['mCherry', 'CFP'])
ax4.set_xlabel('Time (h)')
ax4.set_ylabel('Concentration (M)')

fig5, ax5 = plt.subplots()
m[m.Signal=='LacI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax5)
m[m.Signal=='TetR in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax5)
ax5.legend(['LacI', 'TetR'])
ax5.set_xlabel('Time (h)')
ax5.set_ylabel('Concentration (M)')


fig6, ax6 = plt.subplots()
m[m.Signal=='AiiA in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax6)
ax6.legend(['AiiA'])
ax6.set_xlabel('Time (h)')
ax6.set_ylabel('Concentration (M)')

fig7, ax7 = plt.subplots()
m[m.Signal=='Extracellular C4'].plot(x='Time', y='Measurement', style='-', ax=ax7)
m[m.Signal=='Extracellular C14'].plot(x='Time', y='Measurement', style='-', ax=ax7)
ax7.legend(['C4', 'C14'])
ax7.set_xlabel('Time (h)')
ax7.set_ylabel('Concentration (M)')

# Continuos signals

In [ ]:
def growth_rate(t):
    return lc.gompertz_growth_rate(t, 0.01, 1, 1, 1)

def biomass(t):
    return lc.gompertz(t, 0.01, 1, 1, 1)

metab = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
genetic_n = lc.GeneticNetwork()

In [ ]:
strain = lc.Strain('Strain', genetic_n, metab)

In [ ]:
# Transcription factors
laci = lc.Regulator(name='LacI', degradation_rate=1.02, init_concentration=10**-7)
tetr = lc.Regulator(name='TetR', degradation_rate=1.02)
ci = lc.Regulator(name='cI', degradation_rate=1.02)

# Signalling molecules
c4 = lc.Regulator(name='C4', diffusion_rate=180)
c14 = lc.Regulator(name='C14', diffusion_rate=126)

genetic_n.add_regulator([tetr, laci, ci, c4, c14])

In [ ]:
# option with producers
rhli = lc.Regulator(name='RhlI', degradation_rate=1.02)
cini = lc.Regulator(name='CinI', degradation_rate=1.02)
genetic_n.add_regulator([rhli, cini])

In [ ]:
aiia = lc.Regulator(name='AiiA', degradation_rate=1.02)
genetic_n.add_regulator(aiia)

In [ ]:
mcherry = lc.Reporter(name='mCherry', degradation_rate=1.02, color='red') 
cyan = lc.Reporter(name='CFP', degradation_rate=1.02, color='blue') 
mven = lc.Reporter(name='mVenus', degradation_rate=1.02, color='yellow') 

genetic_n.add_reporter([mcherry, cyan, mven])

In [ ]:
# option with producers

#First extended node
laci_not_aiia = lc.Hill1(name='Plac', input=laci, output=aiia, alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)
cini_c14 = lc.Producer(name='CinI', enzyme=cini, product=c14, k2=60)
c14_tetr = lc.Hill1(name='Pcin', input=c14, output=[tetr, mcherry], alpha=[2.15*10**-8, 7.35*10**-8], K=3.41*10**-6, n=1.87)

# Second node
tetr_not_ci = lc.Hill1(name='Ptet', input=tetr, output=[ci, mven], alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)

# Third extended node
ci_not_aiia = lc.Hill1(name='PcI', input=ci, output=aiia, alpha=[1.44*10**-4, 1.44*10**-7], K=6.64*10**-8, n=2)
rhli_c4 = lc.Producer(name='RhlI', enzyme=rhli, product=c4, k2=960)
c4_laci = lc.Hill1(name='Prhl', input=c4, output=[laci, cyan], alpha=[0, 2.15*10**-8], K=4.68*10**-5, n=0.64)

genetic_n.add_operator([c4_laci, laci_not_aiia, c14_tetr, ci_not_aiia, tetr_not_ci])
genetic_n.add_impactor([cini_c14, rhli_c4])

In [ ]:
lactonase = lc.Degrader(name='Lactonase', enzyme=aiia, substrate=[c4, c14], Km=[0.00511, 0.00143], k2=[1.35*10**5, 7.28*10**4])
genetic_n.add_impactor(lactonase)

# continuous expression of AiiA, one copy
j23117 = lc.Source(output=[rhli, cini], rate=1.54*10**-8, name='J23117')
genetic_n.add_operator(j23117)

In [ ]:
plt.figure(figsize=(3.5,3.5), dpi=100)
genetic_n.draw() #pos=nx.planar_layout

In [ ]:
sample = lc.Sample(strain=strain)
assay = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='Extended repressilator',
                description='Simulated expression of extended repressilator strain'
                )

In [ ]:
assay.run(substeps=100, mode="track_all")

In [ ]:
m = assay.measurements
fig, ax = plt.subplots()
fig1, ax1 = plt.subplots()
fig2, ax2 = plt.subplots()
fig3, ax3 = plt.subplots()
m[m.Signal=='mCherry in Strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
m[m.Signal=='CFP in Strain'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax)
m[m.Signal=='mVenus in Strain'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax)
m[m.Signal=='LacI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='TetR in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='cI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='C4 in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='C14 in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='RhlI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax3)
m[m.Signal=='CinI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax3)
ax.legend(['mCherry', 'CFP', 'mVenus'])
ax.set_xlabel('Time (h)')
ax.set_ylabel('Concentration (M)')
ax1.legend(['LacI', 'TetR', 'cI'])
ax1.set_xlabel('Time (h)')
ax1.set_ylabel('Concentration (M)')
ax2.legend(['C4', 'C14'])
ax2.set_xlabel('Time (h)')
ax2.set_ylabel('Concentration (M)')
ax3.legend(['RhlI', 'CinI'])
ax3.set_xlabel('Time (h)')
ax3.set_ylabel('Concentration (M)')

fig4, ax4 = plt.subplots()
m[m.Signal=='mCherry in Strain'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax4)
m[m.Signal=='CFP in Strain'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax4)
ax4.legend(['mCherry', 'CFP'])
ax4.set_xlabel('Time (h)')
ax4.set_ylabel('Concentration (M)')

fig5, ax5 = plt.subplots()
m[m.Signal=='LacI in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax5)
m[m.Signal=='TetR in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax5)
ax5.legend(['LacI', 'TetR'])
ax5.set_xlabel('Time (h)')
ax5.set_ylabel('Concentration (M)')

fig6, ax6 = plt.subplots()
m[m.Signal=='AiiA in Strain'].plot(x='Time', y='Measurement', style='-', ax=ax6)
ax6.legend(['AiiA'])
ax6.set_xlabel('Time (h)')
ax6.set_ylabel('Concentration (M)')

fig7, ax7 = plt.subplots()
m[m.Signal=='Extracellular C4'].plot(x='Time', y='Measurement', style='-', ax=ax7)
m[m.Signal=='Extracellular C14'].plot(x='Time', y='Measurement', style='-', ax=ax7)
ax7.legend(['C4', 'C14'])
ax7.set_xlabel('Time (h)')
ax7.set_ylabel('Concentration (M)')